# Creation of Storm Patches with UH values that Exceed 75 m2/s2

### This Jupyter Notebook separates the storm patches that exceed or do not exceed the desired UH threshold, which is 75 m2/s2 in this example.
### The output files only contain the UH values.
### This script requires the index files that were created using a previous script.

First, import packages of interest.

In [4]:
import numpy as np
import xarray as xr
import pandas as pd

1 - Select the climate simulation of interest (current or future climate). 

2 - Create array of months across the simulation time period. WRF CONUS1 simulations span October 2000 through September 2013.

In [5]:
which_climate = 'future'

time_array = pd.date_range('2000-10-01','2013-09-30',freq='MS')[(pd.date_range('2000-10-01','2013-09-30',freq='MS').month==5)]

#time_array = pd.date_range('2000-10-01','2013-09-30',freq='MS')[(pd.date_range('2000-10-01','2013-09-30',freq='MS').month==12)|
#                                                                (pd.date_range('2000-10-01','2013-09-30',freq='MS').month==1)|
#                                                                (pd.date_range('2000-10-01','2013-09-30',freq='MS').month==2)|
#                                                                (pd.date_range('2000-10-01','2013-09-30',freq='MS').month==3)|
#                                                                (pd.date_range('2000-10-01','2013-09-30',freq='MS').month==4)|
#                                                                (pd.date_range('2000-10-01','2013-09-30',freq='MS').month==5)]

1 - Open index files that were previously created for each month in the temporal period of interest.

2 - Open the corresponding storm patches.

3 - Extract the storm patch UH values and filter them by the storm patch indices of patches that exceeded 75 m2/s2.

4 - Create xarray data set of the filtered storm patches for future use.

5 - Save to disk.

In [6]:
data_uh_sev_1 = []

for time in time_array[:]:
    
    data_mask = \
    xr.open_mfdataset(f"/glade/scratch/molina/WRF_CONUS1_derived/storm_envs/{which_climate}_conus1_indx75_{time.strftime('%Y')}{time.strftime('%m')}.nc",
                      combine='by_coords')
    data_temp = \
    xr.open_mfdataset(f"/glade/scratch/molina/WRF_CONUS1_derived/storm_envs/{which_climate}_conus1_{time.strftime('%Y')}{time.strftime('%m')}*.nc",
                      combine='by_coords')
    
    
    data_uh_sev_1.append(data_temp.uh_grid.values[data_mask.grid,:,:])
    
    break

data_uh_sev_1_patches = [j for i in data_uh_sev_1 for j in i]

data_assemble = xr.Dataset({
    'uhel_sev_1':(['patch','y','x'], np.array(data_uh_sev_1_patches)),
})

data_assemble.to_netcdf(f"/glade/scratch/molina/WRF_CONUS1_derived/storm_envs/{which_climate}_uh75patchesUH_{time.strftime('%m')}.nc")